# Code from BASS Assignment

This module is solely for running the selected stocks over a given time period to derive the basic metrics for the respective security when compared with a benchmark

In [1]:
!pip install yfinance

In [2]:
# Import relevant libraries
import pandas as pd
import numpy as np
import yfinance as yf
import math

In [7]:
all_stocks = {
    "Consumer Goods": {
        "The Estée Lauder Companies Inc.": "EL",
        "Target Corporation": "TGT",
        "The Home Depot Inc.": "HD",
    },
    "Technology": {
        "Apple Inc.": "AAPL",
        "Facebook": "META",
        "Amazon": "AMZN"
    },
    "Healthcare and Biotech": {
        "Pfizer Inc.": "PFE",
        "Merck & Co., Inc.": "MRK",
        "AbbVie Inc.": "ABBV"
    },
    "Renewable Energy": {
        "Brookfield Renewable Partners L.P.": "BEP",
        "SunPower Corporation": "SPWR",
        "Ormat Technologies": "ORA"
    },
    "Real Estate": {
        "Prologis": "PLD",
        "Simon Property Group": "SPG",
        "Equinix": "EQIX",
    },
    "Streaming": {
        "Netflix Inc.": "NFLX",
        "Walt Disney Co": "DIS",
        "Comcast Corporation": "CMCSA"
    }
}

In [4]:
benchmark_symbol = "SPY"
start = "2010-01-01"
end = "2016-12-31"

In [5]:
def BASS(stock_symbol, benchmark_symbol, start, end):
  df = pd.DataFrame()
  df['benchmark'] = yf.Ticker(benchmark_symbol).history(start=start, end=end).Close
  df['stock'] = yf.Ticker(stock_symbol).history(start=start, end=end).Close
  df['stock_returns'] = df['stock'].pct_change()
  df['benchmark_returns'] = df['benchmark'].pct_change()

  df = df.dropna()
  if len(df) == 0:
    raise Exception(f'ERROR:yfinance:{stock}: No timezone found, symbol may be delisted')

  # Beta calculations
  beta = round(
    df['benchmark_returns'].cov(df['stock_returns']) / df['benchmark_returns'].var(),
    2
  )

  # Calculate Alpha
  benchmark_yearly_returns = df['benchmark_returns'].mean() * 252
  stock_yearly_returns = df['stock_returns'].mean() * 252
  alpha = round(
    (stock_yearly_returns - beta * benchmark_yearly_returns) * 100,
    2
  )

  # Calculate Standard Deviation of stock
  std_dev = round(
    df['stock_returns'].std() * 100,
    2
  )

  # Calculate Sharpe Ratio of stock
  # SR = Mu / Sigma
  avg_returns = df['stock_returns'].mean()
  std = df['stock_returns'].std()
  daily_SR = avg_returns / std
  annual_SR = round(
    daily_SR * (252**0.5),
    2
  )
  return beta, alpha, std_dev, annual_SR

In [8]:
# Calculate the BASS for each stock
bass_dictionary = {
    "start_date": [],
    "end_date": [],
    "industry": [],
    "stock": [],
    "symbol": [],
    "beta": [],
    "alpha": [],
    "std_dev": [],
    "annual_SR": []
}

for industry, stocks in all_stocks.items():
  if stocks == {}:
    continue
  else:
    for stock, stock_symbol in stocks.items():
      try:
        beta, alpha, std_dev, annual_SR = BASS(stock_symbol, benchmark_symbol, start, end)
        bass_dictionary['start_date'].append(start)
        bass_dictionary['end_date'].append(end)
        bass_dictionary['industry'].append(industry)
        bass_dictionary['stock'].append(stock)
        bass_dictionary['symbol'].append(stock_symbol)
        bass_dictionary['beta'].append(beta)
        bass_dictionary['alpha'].append(alpha)
        bass_dictionary['std_dev'].append(std_dev)
        bass_dictionary['annual_SR'].append(annual_SR)
      except Exception as e:
        print(f"No Data for stock: {stock}")

# Create a dataframe to house all the BASS data for the various stocks in STI Index
bass = pd.DataFrame(bass_dictionary, columns = bass_dictionary.keys())

# Sort the data by order of Sharpe Ratio
bass = bass.sort_values(by=['industry', 'beta'], ascending=False).reset_index(drop=True)
bass

,start_date,end_date,industry,stock,symbol,beta,alpha,std_dev,annual_SR
0,2010-01-01,2016-12-31,Technology,Amazon,AMZN,1.11,15.49,2.04,0.92
1,2010-01-01,2016-12-31,Technology,Facebook,META,1.03,16.54,2.53,0.79
2,2010-01-01,2016-12-31,Technology,Apple Inc.,AAPL,0.95,11.53,1.65,0.91
3,2010-01-01,2016-12-31,Streaming,Netflix Inc.,NFLX,1.04,42.67,3.61,0.98
4,2010-01-01,2016-12-31,Streaming,Walt Disney Co,DIS,1.04,7.07,1.36,0.95
5,2010-01-01,2016-12-31,Streaming,Comcast Corporation,CMCSA,0.98,11.58,1.37,1.11
6,2010-01-01,2016-12-31,Renewable Energy,SunPower Corporation,SPWR,1.84,-21.33,4.13,0.04
7,2010-01-01,2016-12-31,Renewable Energy,Ormat Technologies,ORA,1.11,-4.05,1.92,0.34
8,2010-01-01,2016-12-31,Renewable Energy,Brookfield Renewable Partners L.P.,BEP,0.65,4.44,1.51,0.54
9,2010-01-01,2016-12-31,Real Estate,Prologis,PLD,1.30,0.83,1.70,0.65
